In [1]:
import numpy as np
import copy
from scipy.sparse import csr_matrix

In [2]:
def isvalidmove(s, istart, iend):
    if len(s[istart]) == 0: return False
    if len(s[iend]) == 0: return True
    if s[istart][-1]<s[iend][-1]: 
        return True
    else:
        return False

In [3]:
def generateHanoi6status(n):
    res = {}
    valid_move = {i: [] for i in range(n)}
    res[0] = [[6,5,4,3,2,1],[],[],[],[],[]]
    i = 1
    layer = [0]
    while i<n and len(layer) != 0:
        next_layer = []
        for curr_idx in layer:
            curr_status = res[curr_idx]
            for start, end in [(0,1),(1,0),(1,2),(2,1),(2,3),(3,2),(3,4),(4,3),(4,5),(5,4)]:
                if isvalidmove(curr_status, start, end):
                    temp = copy.deepcopy(curr_status)
                    temp[end].append(temp[start].pop()) 
                    if temp not in res.values():
                        res[i] = temp
                        next_layer.append(i)
                        valid_move[curr_idx].append(i)
                        valid_move[i].append(curr_idx)
                        i += 1
                        if i % 1000 == 0: print(i)
        layer = next_layer
    return [res,valid_move]

In [4]:
def computecenterHanoi6(s):
    weightedsum = 0.0
    for i in range(6):
        weightedsum += sum(s[i]) * i
    return weightedsum / 21.0

In [5]:
def computeprobHanoi6(n,T,valid_move):
    i, j, v = [], [], []
    for ii in range(n):
        if ii % 2000 == 0: print(ii)
        for jj in valid_move[ii]:
            i.append(jj)
            j.append(ii)
            v.append(1.0/float(len(valid_move[ii])))
    i, j, v = np.array(i), np.array(j), np.array(v)
    A = csr_matrix((v,(i,j)),shape=(n,n))
    #res = np.concatenate((np.array([1.0]),[0]*(n-1)), axis=0)
    res = csr_matrix(([1],([0],[0])),shape=(n,1))
    print('multiply matrices')
    for k in range(T):
        if k % 20 == 0: print(k)
        res = A * res
    return res

In [6]:
# Main Hanoi6
n = 6**6
T = 256
print('generate status...')
status, valid_move = generateHanoi6status(n)

generate status...
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000


In [7]:
print('generate centers...')
center = [[computecenterHanoi6(status[i]) for i in range(n)]]

generate centers...


In [8]:
print('generate p...')
p = computeprobHanoi6(n,T,valid_move).todense()

generate p...
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
40000
42000
44000
46000
multiply matrices
0
20
40
60
80
100
120
140
160
180
200
220
240


In [10]:
print('compute mean and variance...')
mean = np.dot(center, p)
variance = np.dot(np.square(center-mean), p)
print("%.10f" % mean)
print("%.10f" % variance)
#for i in range(n):
#    print(i, status[i], center[i])

compute mean and variance...
0.7655142451
0.2201701017
